In [1]:
import sys
sys.path.insert(0, './algorithms/alpha_beta')
sys.path.insert(0, './algorithms/dqn')
sys.path.insert(0, './algorithms/stockfish')

In [2]:
from algorithms.MyChessBoard import MyChessBoard
from algorithms.alpha_beta.Alpha_Beta import find_move as alpha_beta_find_move
from algorithms.dqn.dqn_bot import find_move as dqn_find_move
from algorithms.dqn.dqn_sf_bot import find_move as dqn_vs_sf_find_move
from algorithms.stockfish.stockfish_bot import find_move as stockfish_find_move

In [3]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten,\
     Input,BatchNormalization, Dropout

2021-12-16 17:35:20.062288: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-16 17:35:20.062335: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
STATE_SHAPE = (65, )
NB_ACTIONS = 4096


In [5]:
model_dqn = Sequential()
model_dqn.add(Input((1, ) + STATE_SHAPE))
model_dqn.add(Flatten())
model_dqn.add(Dense(128, activation='relu'))
model_dqn.add(Dropout(0.2))
model_dqn.add(BatchNormalization())
model_dqn.add(Dense(128, activation='relu'))
model_dqn.add(Dropout(0.2))
model_dqn.add(BatchNormalization())
model_dqn.add(Dense(NB_ACTIONS))
model_dqn.add(Activation('linear'))
model_dqn.load_weights('chess_dqn_model.h5')


2021-12-16 17:35:22.157580: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-16 17:35:22.157606: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-16 17:35:22.157673: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LoG): /proc/driver/nvidia/version does not exist
2021-12-16 17:35:22.157915: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
model_dqn_sf = Sequential()
model_dqn_sf.add(Input((1, ) + STATE_SHAPE))
model_dqn_sf.add(Flatten())
model_dqn_sf.add(Dense(128, activation='relu'))
model_dqn_sf.add(Dropout(0.2))
model_dqn_sf.add(BatchNormalization())
model_dqn_sf.add(Dense(128, activation='relu'))
model_dqn_sf.add(Dropout(0.2))
model_dqn_sf.add(BatchNormalization())
model_dqn_sf.add(Dense(NB_ACTIONS))
model_dqn_sf.add(Activation('linear'))
model_dqn_sf.load_weights('chess_dqn_vs_sf_model.h5')

In [7]:
board = MyChessBoard()

In [8]:
board.convert_board_to_int()

array([[ -40.,  -20.,  -30.,  -50., -900.,  -30.,  -20.,  -40.],
       [ -10.,  -10.,  -10.,  -10.,  -10.,  -10.,  -10.,  -10.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
       [  10.,   10.,   10.,   10.,   10.,   10.,   10.,   10.],
       [  40.,   20.,   30.,   50.,  900.,   30.,   20.,   40.]])

In [9]:
state = board.get_state()
model_dqn.predict(state.reshape((1, 1) + STATE_SHAPE)).reshape(-1, )

2021-12-16 17:35:23.127915: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-16 17:35:23.152426: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 1999965000 Hz


array([-4108.706 , -3328.7566, -3545.724 , ..., -3909.216 , -3554.2666,
       -4078.8809], dtype=float32)

In [10]:
dqn_find_move(board, model_dqn)

Move.from_uci('b1c3')

In [11]:
name = ["Alpha_Beta", "DQN", "DQN_sf", "Stockfish"]
find = [alpha_beta_find_move, dqn_find_move, dqn_vs_sf_find_move, stockfish_find_move]

result = np.zeros((4, 4))

In [13]:
com1 = 1
num_battle_with_com1 = 1000

com2 = 2
steps = []
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        num_step = 0
        while not done:
            num_step += 1
            board.push(find[com1](board, model_dqn))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
            board.push(find[com2](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            num_step += 1
            board.push(find[com2](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
            board.push(find[com1](board, model_dqn))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True

1 vs 2 : 0
fivefold repetition
1 vs 2 : 1
fivefold repetition
1 vs 2 : 2
fivefold repetition
1 vs 2 : 3
fivefold repetition
1 vs 2 : 4
fivefold repetition
1 vs 2 : 5
fivefold repetition
1 vs 2 : 6
fivefold repetition
1 vs 2 : 7
fivefold repetition
1 vs 2 : 8
fivefold repetition
1 vs 2 : 9
fivefold repetition
1 vs 2 : 10
fivefold repetition
1 vs 2 : 11
fivefold repetition
1 vs 2 : 12
fivefold repetition
1 vs 2 : 13
fivefold repetition
1 vs 2 : 14
fivefold repetition
1 vs 2 : 15
fivefold repetition
1 vs 2 : 16
fivefold repetition
1 vs 2 : 17


KeyboardInterrupt: 

In [11]:
num_battle_with_com1 = 50
com1 = 0

com2 = 1
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        while not done:
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
            board.push(find[com2](board, model_dqn))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            board.push(find[com2](board, model_dqn))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
            
com2 = 2
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        while not done:
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
            board.push(find[com2](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            board.push(find[com2](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True

com2 = 3
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        while not done:
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True



0 vs 1 : 0


KeyboardInterrupt: 

In [ ]:
com1 = 1
num_battle_with_com1 = 1000

com2 = 2
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        while not done:
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
            board.push(find[com2](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            board.push(find[com2](board, model_dqn_sf))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True

com2 = 3
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        while not done:
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True

In [ ]:
com1 = 2
num_battle_with_com1 = 1000

com2 = 3
for i in range (num_battle_with_com1):
    print(com1, "vs", com2, ":", i)
    board = MyChessBoard()
    done = False
    if i < num_battle_with_com1/2:
        while not done:
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
    else:
        while not done:
            board.push(find[com2](board))
            if board.is_checkmate(): 
                result[com2, com1] += 1
                done = True
            if board.is_draw(): 
                done = True
            board.push(find[com1](board))
            if board.is_checkmate(): 
                result[com1, com2] += 1
                done = True
            if board.is_draw(): 
                done = True

In [ ]:
result